# Классификатор справок

**Цель работы** — разработать решение для классификации медицинских справок для последующей интеграции в готовый микросервис по их OCR-распознаванию. Классификатор на данном этапе должен определять, 405 форма справки или же нет, в зависимости от чего будет решаться, передавать ли изображение на распознавание.

Предоставлено различное количество справок 5 типов:

* справки с Госуслуг
* 402 форма
* 406 форма
* 448 форма
* 405 форма (требуемая к распознаванию)

**Ход работы**

Для начала определимся с методом получения векторных представлений, эмбеддингов входящих изображений, подготовим их.

Далее выберем непосредственно классификатор, обучим его на имеющихся данных. После чего в итоге оценим качество его работы на тестовой выборке.
 
Таким образом работа будет состоять из следующих этапов:

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Получение-эмбеддингов-изображений" data-toc-modified-id="Получение-эмбеддингов-изображений-1">Получение эмбеддингов изображений</a></span></li><li><span><a href="#Обучение-классификатора" data-toc-modified-id="Обучение-классификатора-2">Обучение классификатора</a></span></li><li><span><a href="#Проверка-качества-на-тестовых-данных" data-toc-modified-id="Проверка-качества-на-тестовых-данных-3">Проверка качества на тестовых данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4">Вывод</a></span></li></ul></div>

## Получение эмбеддингов изображений

In [1]:
import os
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
import plotly.io as pio
from plotly.figure_factory import create_annotated_heatmap

from sklearn.metrics import \
    f1_score, \
    roc_auc_score, \
    accuracy_score, \
    precision_score, \
    recall_score, \
    confusion_matrix
from sklearn.model_selection import train_test_split

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from transformers import AutoModel

from skorch.classifier import NeuralNetBinaryClassifier
from skorch.callbacks import EpochScoring, EarlyStopping, LRScheduler

In [2]:
HOME = 'C:\\Users\\darve\\Pet_projects\\donorsearch_screenshots'
pio.renderers.default = "notebook_connected"
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.use_deterministic_algorithms(True)
SEED = 345678
set_seed(SEED)

Для начала соберём датафрейм с путями к файлам и бинарным таргетом для получения эмбеддингов фотографий и последующей классификации.

In [3]:
subfolders = ['402', '405', '406', '448', 'gos']
images = []
targets = []

for subfolder in subfolders:
    path = os.path.join(HOME, subfolder)
    for file in os.listdir(path):
        if file.endswith('.jpg'):
            images.append(os.path.join(subfolder, file))
            targets.append(int(subfolder == '405'))

ocr = pd.DataFrame({'images': images, 'is_405': targets})
ocr.sample(10, random_state=SEED)


,images,is_405
7,402\15df545bfeae45b2bae756282a8cb7ef11.large.jpg,0
231,448\dc11323beb2d403382ebb9483965dda3.jpg,0
309,gos\b541f70e3b4444b494dcf433a2730155.jpg,0
203,448\708543765e494e55b041a3b94517d74a.jpg,0
200,448\621433877a3643969f3c702947e56527.jpg,0
331,gos\e5e68b1e804e4694a000fb58141d1d83.jpg,0
116,405\2cb625f02fcb4e01abebb8855ae9eade.jpg,1
141,405\94b271775ec04796a10e65a6cbb45370.jpg,1
303,gos\a0d4ed12d90746b68c9e7ffe6cd0a3e4.jpg,0
294,gos\7d7b928a524a4a1f9a1281349a33b760.jpg,0


Переопределим класс Dataset из PyTorch для подгрузки изображений и напишем функцию для генерации эмбеддингов. Используем чуть больший размер изображения при масштабировании с помощью **Resize()**, чем при его окончательной обрезке, для сохранения большего количества информации.

In [4]:
class DFDataset(Dataset):
    def __init__(self, df, dir, transform=None):
        self.df = df
        self.dir = dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        file = row.images
        path = os.path.join(self.dir, file)
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

def obtaining_img_embeddings(df, dir, model, batch_size):
    preprocess = transforms.Compose([
        transforms.Resize(384),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5]
            )
        ])
    dataset = DFDataset(df, dir, transform=preprocess)
    loader = DataLoader(dataset, batch_size=batch_size)
    image_embeddings = []

    with torch.no_grad():
        for batch in loader:
            output = model(batch)
            image_embeddings.append(output[0][:,0,:].numpy())
            
    return np.concatenate(image_embeddings)

Эмбеддинги получим с помощью предобученной BEiT-модели (бертоподобного ViT) от Microsoft — https://huggingface.co/google/vit-base-patch16-224, во время экспериментов она лучше себя показала для данной задачи.

In [5]:
beit = AutoModel.from_pretrained('microsoft/beit-base-patch16-224-pt22k-ft22k')
with open(os.path.join(HOME, 'beit_encoder.pkl'), 'wb') as f:
    pickle.dump(beit, f)
with open(os.path.join(HOME, 'beit_encoder.pkl'), 'rb') as f:
    beit = pickle.load(f)

embeddings = obtaining_img_embeddings(
    ocr,
    HOME,
    beit,
    128
    )
print(
    'Размер эмбеддингов изображений: ',
    embeddings.shape
    )

Some weights of the model checkpoint at microsoft/beit-base-patch16-224-pt22k-ft22k were not used when initializing BeitModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BeitModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BeitModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Размер эмбеддингов изображений:  (342, 768)


Окончательно подготовим данные к обучению.

In [6]:
target = ocr.is_405
train_features, test_features, train_target, test_target = (
    train_test_split(
        embeddings,
        target,
        test_size=.3,
        stratify=target,
        random_state=SEED
        )
    )
print(
    'Размеры тренировочной и тестовой выборок с эмбеддингами: ',
    train_features.shape,
    test_features.shape,
    '\n',
    'Размеры тренировочной и тестовой выборок с целевым признаком: ',
    train_target.shape,
    test_target.shape,
    sep=''
    )

Размеры тренировочной и тестовой выборок с эмбеддингами: (239, 768)(103, 768)
Размеры тренировочной и тестовой выборок с целевым признаком: (239,)(103,)


In [7]:
f_train = torch.FloatTensor(train_features)
f_test = torch.FloatTensor(test_features)

t_train = torch.FloatTensor(train_target.values)
t_test = torch.FloatTensor(test_target.values)

## Обучение классификатора

В качестве классификатора используем обычную FFNN c одним скрытым слоем на базе бинарного классификатора из библиотеки Skorch. Значения различных гиперпараметров были подобраны ранее с помощью кросс-валидации.

Ориентироваться при обучении в первую очередь будем на F1-меру, так как на данном этапе работы всего микросервиса нам одинаково важно как то, чтобы наибольшее количество 405-ых справок попало дальше на распознавание, так и то, чтобы наименьше возможное количество других справок в данном случае бесполезно нагружало наш сервис.

In [8]:
class Net(nn.Module):
    def __init__(
        self,
        n_hidden_neurons=138,
        n_in_neurons=768,
        n_out_neurons=1
        ):
        super().__init__()

        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons)
        self.bn = nn.BatchNorm1d(n_hidden_neurons)
        self.act = nn.ELU()
        self.fc2 = nn.Linear(n_hidden_neurons, n_out_neurons)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn(x)
        x = self.act(x)
        x = self.fc2(x)
        return x

net = NeuralNetBinaryClassifier(
    module=Net,
    verbose=0,
    lr=5e-2,
    batch_size=-1,
    max_epochs=100,
    optimizer=optim.RAdam,
    optimizer__eps=1e-06,
    callbacks=[
        EpochScoring(
            scoring='f1',
            lower_is_better=False,
            name='F1'
            ),
        EarlyStopping(
            lower_is_better=False,
            monitor='F1',
            patience=40,
            load_best=True
            ),
        LRScheduler(
            policy=optim.lr_scheduler.CosineAnnealingWarmRestarts,
            monitor='F1',
            T_0=10
            )
        ]
    )

In [9]:
net.fit(f_train, t_train)
with open(os.path.join(HOME, 'skorch_ffnn_classifier.pkl'), 'wb') as f:
    pickle.dump(net, f)
with open(os.path.join(HOME, 'skorch_ffnn_classifier.pkl'), 'rb') as f:
    net = pickle.load(f)

## Проверка качества на тестовых данных

Посчитаем отдельно как все типы прогнозов нашей модели, так и различные классификационные метрики на тестовой выборке.

In [10]:
preds = net.predict(f_test)
conf_mx = pd.DataFrame(confusion_matrix(t_test, preds))

fig = create_annotated_heatmap(
    conf_mx.to_numpy().round(3),
    x = ['0', '1'],
    y = ['1', '0'],
    colorscale='Bluered',
    font_colors=['white']
    ).update_layout(
        font_size=15,
        width=1000,
        height=500,
        xaxis_title='Предсказания',
        yaxis_title='Истинные значения',
        title=dict(
            text='Марица ошибок на тестовых данных',
            x=.5,
            y=.95
            )
        ).update_xaxes(title_standoff=5)
fig.show()
print()
print(
    'Precision на тестовых данных',
    precision_score(t_test, preds).round(3)
    )
print(
    'Recall на тестовых данных',
    recall_score(t_test, preds).round(3)
    )
print(
    'F-1 на тестовых данных',
    f1_score(t_test, preds).round(3)
    )
print(
    'Accuracy на тестовых данных',
    accuracy_score(t_test, preds).round(3)
    )
print(
    'AUC-ROC на тестовых данных',
    roc_auc_score(
        t_test,
        net.predict_proba(f_test)[:, 1]
        ).round(3)
    )


Precision на тестовых данных 0.955
Recall на тестовых данных 0.913
F-1 на тестовых данных 0.933
Accuracy на тестовых данных 0.971
AUC-ROC на тестовых данных 0.997


## Вывод

Разработанный классификатор достаточно хорошо справляется с распознаванием справок 405 формы относительно остальных в имеющихся данных, показатели всех метрик выше 0.9. Однако в дальнейшем может потребоваться его доработка для мультиклассификации уже по всем типам справок.